### This is the 2nd codefile of the project, after data-gathering using PRAWs
#### See ./000-reddit-pulls.py


In [1]:
#import numpy as np 
    #maybe
    #tim always does (for .arange() ??)
import pandas as pd


from os import listdir
from os.path import isfile, join

### My Subreddits:
* r/explainlikeimfive (referred to as sub1 below)
* r/nostupidquestions (referred to as sub2 below)

In [2]:
#2 - 
#read in the saved, large quantity of csv's for each chosen subreddit, 
#to get 2 separate ENORMOUS DFs (1 per subreddit)

#Current efficiency issue:
#for future refactoring!
#on 2nd+ runs, this will re-slurp the "nodupes.csv" files that I create later

mypath = "../data/"
many_files = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
    #attrib
    #https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

#read in & create dirty dataframe for sub1
sub1_files = [g for g in many_files if "sub1" in g]
for file in sub1_files:
    #attrib
    #https://mungingdata.com/pandas/read-multiple-csv-pandas-dataframe/
    df_sub1_long = pd.concat((pd.read_csv(i) for i in sub1_files))        
   
#read in & create dirty dataframe for sub1 
sub2_files = [h for h in many_files if "sub2" in h]
for file in sub2_files:
    #attrib
    #https://mungingdata.com/pandas/read-multiple-csv-pandas-dataframe/
    df_sub2_long = pd.concat((pd.read_csv(j) for j in sub2_files))        
    
    


In [21]:
df_sub1_long.shape

(10962, 6)

In [3]:
df_sub1_long.head()

,Unnamed: 0,created_utc,title,self_text,subreddit,Unnamed: 0.1
0,0,1.697390e+09,eli5 could people from the last century compre...,Lets say someone in 1990 plays some Super Mari...,explainlikeimfive,NaN
1,1,1.697390e+09,Eli5 Home EV charging,Don’t know anything about EV charging\n\nWhat ...,explainlikeimfive,NaN
2,2,1.697390e+09,ELI5: why are we potentially able to hold our ...,I understand that it's the lack of oxygen that...,explainlikeimfive,NaN
3,3,1.697389e+09,ELI5: Acquiring my car’s title registration?,"Hey everyone, first time truly posting so bear...",explainlikeimfive,NaN
4,4,1.697389e+09,eli5- engine & cabin air filters,ELI5 - why does the auto mechanic always try t...,explainlikeimfive,NaN


In [4]:
df_sub2_long.shape

(6905, 6)

In [5]:
#3.0 - remove the empty posts, just in case they sneaked in thru the pull code's culling
df_sub1_long = df_sub1_long[ df_sub1_long['self_text'] != '']
df_sub2_long = df_sub2_long[ df_sub2_long['self_text'] != '']


In [6]:
#which they didn't
df_sub1_long.shape

(10962, 6)

In [7]:
df_sub2_long.shape

(6905, 6)

In [24]:
#3.1 remove duplicates from both subreddits' data
    #"" TIP: You can use the created_utc attribute of a post to keep track of the timestamp and ensure non-overlapping pulls. The created_utc attribute represents the post's creation time in UTC.""
    #info this might have saved me the de-duping work

#attrib: Rafi for syntax of dropping dupes
df_sub1_short = df_sub1_long.drop_duplicates()
df_sub2_short = df_sub2_long.drop_duplicates()

In [27]:
print("size of eli5 data w/o dupes: ",df_sub1_short.shape)
print("size of nsq w/o dupes: ",df_sub2_short.shape)

print("total no-dupe rows: ", df_sub1_short.shape[0]+df_sub2_short.shape[0])


size of eli5 data w/o dupes:  (6966, 6)
size of nsq w/o dupes:  (6140, 6)
total rows:  13106


**info:** I have 13106 total non-dupe rows/posts over both together

In [37]:
#3.7 - write each trimmed DF to a csv, just for backups, pre-join
df_sub1_short.to_csv(mypath+'sub1_nodupe.csv')
df_sub2_short.to_csv(mypath+'sub2_nodupe.csv')


In [38]:
# #I discovered an extra erroneous column in this output data, 
# #so I'm removing that. It might be recreated if I rerun anything.
# #currently, this cell doesn't need rerunning
# #df_sub1_short.columns
#     # lisapaul@malumeum code $grep Unnamed ../data/*.csv
#     # ../data/sub1_nodupe.csv:,Unnamed: 0,created_utc,title,self_text,subreddit,Unnamed: 0.1
#     # ../data/sub2_nodupe.csv:,Unnamed: 0,created_utc,title,self_text,subreddit,Unnamed: 0.1
# df_sub1_short.drop(columns=['Unnamed: 0.1'], inplace=True)
# df_sub2_short.drop(columns=['Unnamed: 0.1'], inplace=True)

In [36]:
df_sub1_short.columns

Index(['Unnamed: 0', 'created_utc', 'title', 'self_text', 'subreddit'], dtype='object')

In [44]:
#3.8
#column name planning for after joined, double-tall DF

#this is just because I ran a couple things out of order once
df_sub1_short.drop(columns=['is_eli5'], inplace=True)
df_sub2_short.drop(columns=['is_eli5'], inplace=True)


#attrib nolan: concat through same cols
#& reminder why/how, attrib Juston
#Yes combine in same cols (not double wide DF), BUT WITH just one extra col, binarized for which subreddit

#first attempt
#df_sub1_short['is_eli5'] = 1
#rewritten, attrib ChatGPT, due to SettingWithCopyWarning

df_sub1_short['is_eli5'] = None
df_sub1_short.loc[:, 'is_eli5'] = 1

df_sub1_short.columns
#df_sub1_short.head()

df_sub2_short['is_eli5'] = None
df_sub2_short.loc[:, 'is_eli5'] = 0

df_sub2_short.columns
#df_sub2_short.head(2)

/var/folders/3n/chlf84kj2yzdb249ph24n5tm0000gn/T/ipykernel_1112/189018337.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub1_short['is_eli5'] = None
/var/folders/3n/chlf84kj2yzdb249ph24n5tm0000gn/T/ipykernel_1112/189018337.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub2_short['is_eli5'] = None


,Unnamed: 0,created_utc,title,self_text,subreddit,is_eli5
0,1,1.697460e+09,How do people make memes like this?,https://reddit.com/r/shitposting/s/8YOkZrYh5d ...,NoStupidQuestions,0
1,2,1.697460e+09,Why do so many people freak out at the airport...,I feel like almost every day I see a video of ...,NoStupidQuestions,0


In [47]:
#4 join both subreddits' DFs, 
    
#attrib: Kiersten
    #axis 0 = stack //vertically//
df_both_subs = pd.concat([df_sub1_short, df_sub2_short], axis = 0)



df_both_subs.head()

,Unnamed: 0,created_utc,title,self_text,subreddit,is_eli5
0,0,1.697390e+09,eli5 could people from the last century compre...,Lets say someone in 1990 plays some Super Mari...,explainlikeimfive,1
1,1,1.697390e+09,Eli5 Home EV charging,Don’t know anything about EV charging\n\nWhat ...,explainlikeimfive,1
2,2,1.697390e+09,ELI5: why are we potentially able to hold our ...,I understand that it's the lack of oxygen that...,explainlikeimfive,1
3,3,1.697389e+09,ELI5: Acquiring my car’s title registration?,"Hey everyone, first time truly posting so bear...",explainlikeimfive,1
4,4,1.697389e+09,eli5- engine & cabin air filters,ELI5 - why does the auto mechanic always try t...,explainlikeimfive,1


In [48]:
#4.9 - write the combo DF to a csv, just in case
df_both_subs.to_csv(mypath+'both_subs.csv')

#length of new file seems correct
# -rw-r--r--   1 lisapaul  staff  2910337 Oct 19 21:17 sub1_nodupe.csv
# -rw-r--r--   1 lisapaul  staff  3200120 Oct 19 21:17 sub2_nodupe.csv
# drwxr-xr-x  22 lisapaul  staff      704 Oct 19 21:28 .
# -rw-r--r--   1 lisapaul  staff  6124339 Oct 19 21:28 both_subs.csv


### Here ends this notebook